In [64]:
from last_stretch import shortest_path
from shortest_path import filter_categories, get_position

from locations import locations

import re
file = open("auto_teleport_loop.gpc", "w")
dr = "SWI_RIGHT"
dl = "SWI_LEFT"
du = "SWI_UP"
dd = "SWI_DOWN"
plus = "SWI_PLUS"
min = "SWI_MINUS"
ab = "SWI_A"
yb = "SWI_Y"
bb = "SWI_B"
xb = "SWI_X"
zr = "SWI_ZR"
zl = "SWI_ZL"
rb = "SWI_R"
lb = "SWI_L"

# Arguments: controller input
get_val = "get_val"

# Arguments: input, intensity out of 100
set_val = "set_val"

# Argument: Waittime in milliseconds i.e. 1000ms = 1s
wait = "wait"

# Argument: combo to stop
combo_stop = "combo_stop"

# Argument: combo to run
combo_run = "combo_run"

# Arguments 1 selection of LED_1 through LED_4 and a boolean on whether it is on or off
# Update when colors known
set_led = "set_led"

# DISPLAY to screen with print(x_coordinates, y_coordinates, font_size, color, string_variable)
prnt = "print"

# Arguments: controller_input
# Different from get_val because it prevents the game from reading it in as a normal input
event_press = "event_press"

wait_time = [100]

In [65]:
commands = {"get_val: gets an input without disrupting normal game inputs",
            "set_val: inputs a selected button with a percentage argument for how far down the button press is, note: buttons also need unpressed when pressed with this function",
            "wait: waits based on an input value measured in milliseconds",
            "combo_run: invokes a function or 'combo' to run with a combo name as an argument",
            "event_press: gets an input from controller while obstructing the input from being read in by the game"}


In [66]:
def write_command(command_name, input_array, tab_num = 1):
    file.write(f'{'\t' * tab_num}{command_name}(')
    for input in input_array[:-1]:
        file.write(f'{input}, ')
    file.write(f'{input_array[-1]});\n')

In [67]:
def cluster_commands(command_array, array_of_array_input):
    for i, command in enumerate(command_array):
        write_command(command, array_of_array_input[i])

In [68]:
def button_input(button):
    cluster_commands([set_val, wait, set_val, wait], [[button, 100], wait_time, [button, 0], wait_time])

In [69]:
def button_sequence(button_array):
    for button in button_array:
        button_input(button)

In [70]:
def string_to_variable(target_string):
    variable_name = ''
    for char in target_string.lower():
        if char in ' -':
            variable_name += '_'
        elif char in '().': continue
        else:
            variable_name += char
    return variable_name

In [71]:
def convert_array_in_string(input_array):
    array = []
    i = 0
    ph_string = ''
    check = False
    for char in input_array:
        if re.match('[-0-9]', char):
            ph_string += char
            check = True
        elif check == True and re.match('[^-0-9]', input_array):
            array.append(ph_string)
            ph_string = ''
            check = False
        else:
            continue
    return array

In [72]:
def combo_firstline(desired_name):
    file.write(f'combo {desired_name}' + r' {' + '\n')

In [73]:
def end_combo():
    file.write('}\n\n')

In [74]:
# def write_combo(name, command_array, parameter_array_squared):
#     initialize_combo_firstline(name)
#     for i in range(len(command_array)):
        # cluster_commands(command_array[i], parameter_array_squared[i])
    #     button_sequence(parameter_array_squared[i])
    # end_combo()


In [75]:
def map_to_menu():
    combo_firstline("map_to_menu")
    button_sequence([plus, yb, min])
    end_combo()

In [76]:
def menu_selection(category):
    category_index = get_position(category, filter_categories)
    category = string_to_variable(category)
    combo_firstline(f'{category}_map_to_menu')
    write_command(combo_run, ["map_to_menu"])
    dpad_inputs = []
    for i in range(category_index):
        dpad_inputs.append(dd)
    dpad_inputs.append(ab)
    button_sequence(dpad_inputs)
    end_combo()

In [77]:
def create_reused_combos():
    map_to_menu()
    for category in filter_categories[:-1]:
        menu_selection(category)

In [78]:
# def category_teleport_init(category):
    

In [79]:
def coordinates_to_dpad(coordinates):

    x_coordinate_button = dr
    y_coordinate_button = dd
    dpad_array = []

    if coordinates[0] < 0: x_coordinate_button = dl

    if coordinates[1] < 0: y_coordinate_button = du
    
    if coordinates[0] != 0:
        for i in range(abs(coordinates[0])):
            dpad_array.append(x_coordinate_button)

    if coordinates[1] != 0:
        for i in range(abs(coordinates[1])):
            dpad_array.append(y_coordinate_button)
    
    dpad_array.append(ab)
    
    # print(dpad_array)
    # print('\n\n')
    return dpad_array

In [80]:
def location_selection(name, category, coordinates):
    category = string_to_variable(category)
    variable_name = string_to_variable(name)
    combo_firstline(variable_name)
    write_command("combo_run", [f'{category}_map_to_menu'])
    button_presses = coordinates_to_dpad(coordinates)
    button_sequence(button_presses)
    end_combo()

In [83]:
def main():
    create_reused_combos()
    for location in locations[:3]:
        coordinates, name, category  = shortest_path(location)
        location_selection(name, category, coordinates)

In [84]:
main()

***ABSOLUTE SHORTEST PATH FOR: CENTRICO PLAZA***
All Locations
Steps: 0
Coordinates: [0, 0]

***ABSOLUTE SHORTEST PATH FOR: GARE DE LUMIOSE***
All Locations
Steps: 1
Coordinates: [0, 1]

***ABSOLUTE SHORTEST PATH FOR: POKEMON RESEARCH LAB***
All Locations
Steps: 2
Coordinates: [0, 2]

